In [11]:
import numpy as np
import holoviews as hv
hv.extension('matplotlib')
%output fig='svg'

In [12]:
import csv
import pandas as pd
from matplotlib.mlab import csv2rec
from matplotlib.cbook import get_sample_data

In [13]:
with open('position_error_log.csv', 'r', newline='') as error_log:
    reader = csv.reader(error_log)
    header = next(reader)
    
    datadict = {header[i]: [] for i in range(len(header))}
    datadict['Request Number'] = []
    datadict['Cumulative Poses'] = []
    
    request_number, cumulative_poses = 0, 0
    
    for row in reader:
        if row[1] == 1:
            request_number += 1
        
        cumulative_poses += 1
        
        datadict['Request Number'].append(request_number)
            
        for i in range(len(row)):
            datadict[header[i]].append(row[i])


## Define data

In [ ]:
title = ('Calculated IK Solution End Effector Position Error')

# These are the colors that will be used in the plot
color_sequence = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',
                  '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
                  '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',
                  '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5']

# Offsets for degree labels
y_offsets = {'Foreign Languages': 0.5, 'English': -0.5,
             'Communications and Journalism': 0.75,
             'Art and Performance': -0.25, 'Agriculture': 1.25,
             'Social Sciences and History': 0.25, 'Business': -0.75,
             'Math and Statistics': 0.75, 'Architecture': -0.75,
             'Computer Science': 0.75, 'Engineering': -0.25}

# Load the data into a dataframe and us pd.melt to unpivot the degree column
df = pd.DataFrame(datadict)
df = pd.melt(df, var_name='', value_name='conferred')
df['Degree'] = [d.replace('_', ' ').title() for d in df.Degree]

# Define a formatter that works for both bokeh and matplotlib
def percent_format(x):
    try:
        return '{:0.0f}%'.format(x)
    except:
        return '%d%' % x

# Define the value dimensions
value_dim = hv.Dimension('conferred', value_format=percent_format, range=(0, 90))

# Define the dataset
ds = hv.Dataset(df, vdims=[value_dim])
curves = ds.to(hv.Curve, 'year', groupby='Degree').overlay()

# Define a function to get the text annotations
max_year = ds['year'].max()
def get_labels():
     return hv.NdOverlay({deg: hv.Text(max_year, ds[max_year, deg]+y_offsets.get(deg, 0),
                                        deg, halign='left', fontsize=10)(style=dict(color=col))
                            for deg, col in zip(df.Degree.unique(), color_sequence)})

## Display in matplotlib

In [5]:
# Define a callback to define a custom grid along the y-axis and disabling the (ugly) axis spines
def cb(plot, element):
    ax = plot.handles['axis']
    ax.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

# Define various options to adjust the plot
options = hv.Store.options(backend='matplotlib')
options.Curve = hv.Options('plot', show_frame=False, bgcolor='white', labelled=[], show_grid=False,
                           aspect=0.7, show_legend=False, xticks=5, final_hooks=[cb], fig_size=350)
options.Curve = hv.Options('style', color=hv.Cycle(values=color_sequence), linewidth=2)

(curves * get_labels()).relabel(title)

['2.4221429330851914',
 '2.424329180219976',
 '2.4161701869739582',
 '2.3956444973225492',
 '2.3610995265281809',
 '2.3113205201814151',
 '2.245586344340663',
 '2.1637106506855233',
 '2.0660637313118246',
 '1.9535727332064852',
 '1.953677389934974',
 '1.7781691027283535',
 '1.6106817350151377',
 '1.4781325740157472',
 '1.4094643701026406',
 '1.4153134719459315',
 '1.4784496895648027',
 '1.5755703887565788',
 '1.7005761177230696',
 '1.8074182260806271',
 '1.9201262145978897',
 '2.0168163178575154',
 '2.1134588425455574',
 '2.1727270297563313',
 '2.2311246817770289',
 '2.2883899877916569',
 '2.3442921913333454',
 '2.4221364398003979',
 '2.4280922742257474',
 '2.4341092338710992',
 '2.4406518789146623',
 '2.4480807327748413',
 '2.4566221427827983',
 '2.4663494959534584',
 '2.4765367643207656',
 '2.487500877816252',
 '2.4989234313227269',
 '2.5103848685498042',
 '2.5213853331228449',
 '2.5313710535824407',
 '2.5388611740513598',
 '2.5447342844919336',
 '2.5486340987200093',
 '2.55025472907

In [14]:
pd.melt?